In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import glob
import re

%matplotlib inline

In [ ]:
def Gaussian(x, a, Mean, Sigma):
    return a * np.exp(-((x - Mean)**2 / (2 * Sigma**2)))

In [ ]:
def ReadParticleConfig(path):
    with open(path,'r') as F:
        Data = F.read()
    Data = Data.strip()
    FirstRow = Data.split(sep='\n')[0]
    NumberOfAParticles = int(FirstRow.split()[6].strip('|'))
    NumberOfBParticles = int(FirstRow.split()[8].strip('|'))
    ListOfRows = Data.split(sep='\n')[1:]
    if (len(ListOfRows) > 0):
        NumberOfColumns = len(ListOfRows[0].split()) - 2 
    else:
        NumberOfColumns = 0
    APositions = np.empty((NumberOfAParticles, NumberOfColumns))
    BPositions = np.empty((NumberOfBParticles, NumberOfColumns))
    AIndex = 0
    BIndex = 0
    
    for i in range(len(ListOfRows)):
        if (ListOfRows[i].split()[NumberOfColumns + 1] == 'A'):
            for j in range(NumberOfColumns):
                APositions[AIndex,j] = ListOfRows[i].split()[j+1]
            AIndex += 1
        else:
            for j in range(NumberOfColumns):
                BPositions[BIndex,j] = ListOfRows[i].split()[j+1]
            BIndex += 1
    return APositions, BPositions

In [ ]:
def ReadSeries(path):
    with open(path,'r') as F:
        Data = F.read()
    Data = Data.strip()
    ListOfData = Data.split(sep='\n')
    NumberOfEntries = len(ListOfData)
    Series = np.empty(NumberOfEntries)
    for i in range(NumberOfEntries):
        Series[i] = ListOfData[i]
    return Series

In [ ]:
def computeEquilibriumAverage(Series):
    n = 0
    while (n < len(Series)):
        Average = np.mean(Series[n:])
        Deviations = np.abs(Average-Series[n:])
        AverageDeviation = np.mean(Deviations)
        if (np.abs(Average-Series[n]) < AverageDeviation):
            return Average,AverageDeviation,n
        n += 1
    return np.mean(Series[:]), np.mean(np.abs(np.mean(Series[:])-Series[:])), 0

## Plot particle positions

In [ ]:
APositions, BPositions = ReadParticleConfig("data/FinalParticleConfig_N=500_T=2.000000_MinNA=0_MCRuns=1000000.dat")

In [ ]:
Fig, Host = plt.subplots(1,1,squeeze=False,figsize=(5,5),dpi =100)
Host[0,0].plot(APositions[:,0],APositions[:,1],'.',color='b')
Host[0,0].plot(BPositions[:,0],BPositions[:,1],'.',color='darkorange')
Host[0,0].set_xlabel('x')
Host[0,0].set_ylabel('y')
Host[0,0].set_xlim(0,1)
Host[0,0].set_ylim(0,1)

## Series of particle number

In [ ]:
NumberOfASeries = ReadSeries("data/NA_Series_N=500_T=2.000000_MinNA=0_MCRuns=1000000.dat")

In [ ]:
NumberOfAAverage, AverageNumberDeviation, NumberIndex = computeEquilibriumAverage(NumberOfASeries)
print(NumberOfAAverage, AverageNumberDeviation, NumberIndex)

In [ ]:
Fig, Host = plt.subplots(1,1,squeeze=False, figsize = (12,5), dpi = 100)
Host[0,0].plot(NumberOfASeries,'.')
Host[0,0].plot(np.ones(len(NumberOfASeries))*NumberOfAAverage,color='darkorange',label='Gleichgewichts-Mittel')
Host[0,0].plot(np.ones(len(NumberOfASeries))*NumberOfAAverage+AverageNumberDeviation,color='darkorange',alpha= 0.5)
Host[0,0].plot(np.ones(len(NumberOfASeries))*NumberOfAAverage-AverageNumberDeviation,color='darkorange',alpha = 0.5)
Host[0,0].vlines(NumberIndex, min(NumberOfASeries), max(NumberOfASeries), color = 'k', alpha = 0.5)

## Series of potential energy

In [ ]:
PotEnergySeries = ReadSeries("data/PotEnergySeries_N=500_T=2.000000_MinNA=0_MCRuns=1000000_INTERVAL=200.dat")

In [ ]:
PotEnergyAverage, AverageEnergyDeviation, EnergyIndex = computeEquilibriumAverage(PotEnergySeries)
print(PotEnergyAverage, AverageEnergyDeviation, EnergyIndex)

In [ ]:
Fig, Host = plt.subplots(1,1,squeeze=False, figsize = (12,5), dpi = 100)
Host[0,0].plot(PotEnergySeries,'.')
Host[0,0].plot(np.ones(len(PotEnergySeries))*PotEnergyAverage,color='darkorange',label='Gleichgewichts-Mittel')
Host[0,0].plot(np.ones(len(PotEnergySeries))*PotEnergyAverage+AverageEnergyDeviation,color='darkorange',alpha= 0.5)
Host[0,0].plot(np.ones(len(PotEnergySeries))*PotEnergyAverage-AverageEnergyDeviation,color='darkorange',alpha = 0.5)
Host[0,0].vlines(EnergyIndex, min(PotEnergySeries), max(PotEnergySeries), color = 'k', alpha = 0.5)

## Gaussian fit

In [ ]:
def computeHist(Series):
    Min = int(np.min(Series))
    Max = int(np.max(Series))
    yValues = np.zeros(Max - Min + 1)
    xValues = np.arange(Min,Max + 1)
    for Entry in Series:
        yValues[int(Entry)-Min] += 1
    return yValues,xValues

In [ ]:
yValues, xValues = computeHist(NumberOfASeries[NumberIndex:])

In [ ]:
popt, pcov = curve_fit(Gaussian,xValues,yValues,p0=[2000,NumberOfAAverage,AverageNumberDeviation],maxfev=1000000)
print('mean='+str(round(popt[1],6))+'+-'+str(round(popt[2],5))+' , Max: '+str(round(popt[0],3)))

In [ ]:
Fig, Ax = plt.subplots(1,1,squeeze=False,figsize=(5,5))
Ax[0,0].plot(xValues, yValues,'.')
Ax[0,0].plot(xValues,Gaussian(xValues,*popt))